## 情感分布考察

In [1]:
import os
import pandas as pd
import numpy as np
import paddlehub as hub
import imageio

C:\Users\Adydio\anaconda\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [2]:
df = pd.read_csv("zhyzx.csv")

In [3]:
df = df['content']

In [4]:
df

0      我看的最远的地方就是，平安回家！回到妈妈的身旁。父亲是烈士，我重启父亲警号，干了缉毒警。每次...
1                                   喜欢了她七年给她以前唱的歌现在我们在一起
2                                       全书最直的人穿一身基佬紫[大哭]
3      32岁许嵩新专辑《老夫聊发少年狂》主打歌曲《老古董》率先上线，专辑以“老”为主题，收录了包含...
4      毁灭孕育重生［神树］明明有着星系陨落般的荒芜感却澎湃着向死而生的希望［降临］既有辰曦携带万顷...
                             ...                        
460                               我梦寐以求是真爱和自由。这一句歌词太牛逼了。
461    去年高考前几个月出的专辑，我最爱这首。并不是为感情问题悲伤，而是压抑的高中没有情绪宣泄的地方...
462    我再也没有给你留过言再也没有主动找过你再也没有给你打过电话再也没有给你发过短信看见你只会擦肩...
463                                 很喜欢这首歌，虽然我是男的，好像还比较直
464    和发小走在回家的路上，下过雨，地很湿，和她分享了耳机，正在放这首歌。很温馨，无关风月，只是友...
Name: content, Length: 465, dtype: object

In [5]:
# 加载模型
senta = hub.Module(name="senta_bilstm")

[2023-05-13 12:46:39,268] [ WARNING] - The _initialize method in HubModule will soon be deprecated, you can use the __init__() to handle the initialization of the object


In [6]:
test_text = df.tolist()
for i in range(len(test_text)):
    test_text[i] = str(test_text[i])

In [7]:
results = senta.sentiment_classify(texts=test_text, use_gpu=False, batch_size=1)

In [8]:
results_df = pd.DataFrame(results)

In [9]:
df2 = pd.concat([df,results_df],axis=1)

In [14]:
df2.head(50)

,content,text,sentiment_label,sentiment_key,positive_probs,negative_probs
0,我看的最远的地方就是，平安回家！回到妈妈的身旁。父亲是烈士，我重启父亲警号，干了缉毒警。每次...,我看的最远的地方就是，平安回家！回到妈妈的身旁。父亲是烈士，我重启父亲警号，干了缉毒警。每次...,1,positive,0.9652,0.0348
1,喜欢了她七年给她以前唱的歌现在我们在一起,喜欢了她七年给她以前唱的歌现在我们在一起,1,positive,0.9811,0.0189
2,全书最直的人穿一身基佬紫[大哭],全书最直的人穿一身基佬紫[大哭],1,positive,0.9827,0.0173
3,32岁许嵩新专辑《老夫聊发少年狂》主打歌曲《老古董》率先上线，专辑以“老”为主题，收录了包含...,32岁许嵩新专辑《老夫聊发少年狂》主打歌曲《老古董》率先上线，专辑以“老”为主题，收录了包含...,1,positive,0.9840,0.0160
4,毁灭孕育重生［神树］明明有着星系陨落般的荒芜感却澎湃着向死而生的希望［降临］既有辰曦携带万顷...,毁灭孕育重生［神树］明明有着星系陨落般的荒芜感却澎湃着向死而生的希望［降临］既有辰曦携带万顷...,1,positive,0.9945,0.0055
5,今天被暗恋的男孩摸脸颊了！你们能体会那种仿佛要瞬间螺旋升天的喜悦吗!因为今天和他一起去参加同...,今天被暗恋的男孩摸脸颊了！你们能体会那种仿佛要瞬间螺旋升天的喜悦吗!因为今天和他一起去参加同...,1,positive,0.8817,0.1183
6,格聂神山出来的孩子，加油！！！[拜],格聂神山出来的孩子，加油！！！[拜],1,positive,0.9386,0.0614
7,谁说男生不可以有少女情怀呢？我最大的梦想是以后能生个小公主。我要把她打扮得漂漂亮亮的，看其他...,谁说男生不可以有少女情怀呢？我最大的梦想是以后能生个小公主。我要把她打扮得漂漂亮亮的，看其他...,1,positive,0.9951,0.0049
8,异地恋金曲排行榜第三名。[大哭],异地恋金曲排行榜第三名。[大哭],1,positive,0.9924,0.0076
9,我花钱是一个大手大脚的人，平时一有钱就买衣服裤子鞋子，也会买一些演唱会和音乐节的门票，放假了...,我花钱是一个大手大脚的人，平时一有钱就买衣服裤子鞋子，也会买一些演唱会和音乐节的门票，放假了...,1,positive,0.8674,0.1326


In [10]:
df2["new_sentiment_label"] = df2["positive_probs"].map(lambda x: 1 if x>=0.5 else 0)
df2["p"] = df2["positive_probs"].map(lambda x: 0.5 if x>=0 else 0)

In [15]:
#df2 = df2[['userid','timeStr','content','new_sentiment_label','p']]
df2 = df2[['content','positive_probs']]

In [16]:
df2.head()

,content,positive_probs
0,我看的最远的地方就是，平安回家！回到妈妈的身旁。父亲是烈士，我重启父亲警号，干了缉毒警。每次...,0.9652
1,喜欢了她七年给她以前唱的歌现在我们在一起,0.9811
2,全书最直的人穿一身基佬紫[大哭],0.9827
3,32岁许嵩新专辑《老夫聊发少年狂》主打歌曲《老古董》率先上线，专辑以“老”为主题，收录了包含...,0.9840
4,毁灭孕育重生［神树］明明有着星系陨落般的荒芜感却澎湃着向死而生的希望［降临］既有辰曦携带万顷...,0.9945


In [17]:
df2.to_csv("zhyzx_emotion_probs.csv",encoding='utf_8_sig')